In [2]:
import torch
import numpy as np
import cv2
import lightning as L
from tqdm import tqdm
from datasets_all.mnist import SumMNIST
import yaml
from mnist_model_pretrained import LITMNISTModel

class Config:
    def __init__(self, config):
        self.config = config
        for k, v in self.config.items():
            self.__setattr__(k,  v)

config = "configs/red.yaml"

with open(config, "r") as f:
    config = Config(yaml.safe_load(f))
    
dataset_class = {"mnist" : SumMNIST}
val_dataset = dataset_class[config.dataset_type](config.dataset_config, split="val")
valloader = torch.utils.data.dataloader.DataLoader(val_dataset, batch_size=1024, num_workers=4)
model = LITMNISTModel.load_from_checkpoint("../pretrained_models/best_loss_w_pretraining.ckpt")
model.eval()
count = 0
device = torch.device("cuda:0")
metrics = {"acc" : 0}

with torch.no_grad():
    for batch in tqdm(valloader):
        data = batch
        for k, v in batch.items():
            batch[k] = v.to(device)
        output = model.model.validate(batch)
        acc = output['acc']
        metrics['acc'] += acc * len(data['img'])
        count += len(data['img'])
        
acc = metrics['acc'] / count
print(acc)

/home2/keshav06/miniconda3/envs/keshav/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 10/10 [00:02<00:00,  3.87it/s]

tensor(0.9653, device='cuda:0')
